In [5]:
!pip install evaluate
!pip install rouge-score
!pip install scikit-learn
!pip install streamlit
!pip install matplotlib
!pip install seaborn

In [6]:
import json
import evaluate
import os 

None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


In [7]:
# Load evaluation metrics
rouge = evaluate.load('rouge')
bleu = evaluate.load('bleu')
meteor = evaluate.load('meteor')
exact_match = evaluate.load('exact_match')
f1 = evaluate.load('f1')
# bertscore = evaluate.load('bertscore')
# unieval = evaluate.load('unieval')
# geval = evaluate.load('geval')

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/deveshsurve/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/deveshsurve/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/deveshsurve/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [8]:
from openai import OpenAI
import streamlit as st

openai_client = OpenAI(api_key=st.secrets["openai"])

# OpenAI model answer function
def get_answer_openai(question, context):
    response = openai_client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "user", "content": f"Answer the question based on the context:\n\nContext: {context}\n\nQuestion: {question}\nAnswer:"}
        ]
    )
    return response.choices[0].message.content

# Prepare the dataset
questions = []
reference_answers = []
with open('eval_data.jsonl', 'r') as f:
    for line in f:
        data = json.loads(line)
        questions.append(data['prompt'])
        reference_answers.append(data['referenceResponse'])

# Check if GPT answers are already saved
if os.path.exists('gpt_answers.json'):
    with open('gpt_answers.json', 'r') as f:
        gpt_answers = json.load(f)
else:
    # Generate answers using GPT
    gpt_answers = []
    for question in questions:
        answer = get_answer_openai(question, context="")  # You can add context if needed
        gpt_answers.append(answer)

    # Save GPT answers to a file
    with open('gpt_answers.json', 'w') as f:
        json.dump(gpt_answers, f)

# Compare generated answers with reference answers using evaluation metrics
rouge_scores = rouge.compute(predictions=gpt_answers, references=reference_answers)
bleu_scores = bleu.compute(predictions=gpt_answers, references=reference_answers)
meteor_scores = meteor.compute(predictions=gpt_answers, references=reference_answers)
exact_match_scores = exact_match.compute(predictions=gpt_answers, references=reference_answers)
# f1_scores = f1.compute(predictions=gpt_answers, references=reference_answers)


In [9]:
import matplotlib.pyplot as plt

# Extract the scores
rouge_score = rouge_scores['rougeL']
bleu_score = bleu_scores['bleu']
meteor_score = meteor_scores['meteor']
exact_match_score = exact_match_scores['exact_match']

# Create a list of scores and metric names
scores = [rouge_score, bleu_score, meteor_score, exact_match_score]
metrics = ['ROUGE-L', 'BLEU', 'METEOR', 'Exact Match']

# Create a horizontal bar chart
plt.figure(figsize=(10, 6))
plt.barh(metrics, scores, color=['blue', 'orange', 'green', 'red'])
plt.title('Comparison of Evaluation Metrics')
plt.xlabel('Scores')
plt.xlim(0, 1)  # Assuming the scores are between 0 and 1
plt.show()


/var/folders/qm/2m60vf_d5z3b2hf93l6yznfr0000gn/T/ipykernel_6304/2827803265.py:19: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()


In [10]:
plt.show()


/var/folders/qm/2m60vf_d5z3b2hf93l6yznfr0000gn/T/ipykernel_6304/3040330175.py:1: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()
